In [5]:
import pandas as pd
from os import getcwd, path, listdir, chdir

TICKER_PATH = "data\\tickers\\ticker_data"

'c:\\Users\\anton\\Documents\\antoniouaa\\msc_thesis\\data\\tickers\\ticker_data\\data\\tickers\\ticker_data'

In [6]:
def grab_all_tickers():
    chdir(f"..\{TICKER_PATH}")
    current_dir = getcwd()
    files = [f for f in listdir(current_dir) if path.isfile(path.join(current_dir, f))]
        return files

FileNotFoundError: [WinError 3] The system cannot find the path specified: '..\\data\\tickers\\ticker_data'

In [ ]:
df = pd.read_csv("BC.csv", parse_dates=True, header=0)

In [7]:
def calc_moving_averages(df, window=14):
    df_new = df[["close", "high", "low", "open", "volume"]]

    df_ma = df.rolling(window=14).mean()["open"]
    df_std = df.rolling(window=14).std()["open"]

    final_df = df_new.copy()
    final_df["Rolling Mean"] = df_ma
    final_df["Rolling STDDEV"] = df_std
    return final_df

In [ ]:
tickers = grab_all_tickers()
for ticker in tickers:
    temp_df = pd.read_csv(ticker), parse_dates=True, header=0)
    